In [0]:

--select * from bookstore.bronze.customers limit 10;

CREATE OR REPLACE TEMPORARY VIEW customers AS
SELECT * FROM JSON.`s3://bookstorerawdata/dataset/ASSOCIATE/customers-json/`;

select * from customers limit 10;

CREATE OR REPLACE TEMPORARY VIEW parsed_customers AS
SELECT  customer_id
        ,profile
        ,profile_struct
        ,profile_struct.*
from    (
        SELECT  customer_id,profile
                ,from_json(
                  profile
                  ,schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')
                  ) AS profile_struct
        FROM customers
        );

select * from parsed_customers limit 10;

/*
"schema_of_json" --> derives schema details (DDL format) from nested JSON data provided.
"from_json" --> converts JSON string "profile" into "struct" data type with help of schema derived in step1.
*/

DESCRIBE customers; --nested data stored as plain string (profile).
DESCRIBE parsed_customers; --nested data stored as struct data type (profile & address).

-- use colon operator (:) to reference fields in a nested json "string". 
-- But, you cannot use * operator to reference "all" fields; instead you need to reference each field individually by its name.
SELECT customer_id,profile:first_name,profile:address:country 
FROM customers;

-- use dot operator (.) to reference fields in a "struct" data type. 
-- Here, you can use * operator to reference "all" fields.
SELECT customer_id,profile_struct.first_name,profile_struct.address.country
FROM parsed_customers;
SELECT customer_id,profile_struct.*
FROM parsed_customers;
